In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [7]:
# 데이터 로드

train = pd.read_parquet('./train.parquet')

In [8]:

from sklearn.utils import shuffle

tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

C:\Users\soy66\AppData\Local\Temp\ipykernel_12872\892365265.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
C:\Users\soy66\AppData\Local\Temp\ipykernel_12872\892365265.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


(4426522, 12)
(4426522,)
(274695, 12)


In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4426522 entries, 2506587 to 3089975
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   base_hour              int64  
 1   lane_count             int64  
 2   road_rating            int64  
 3   maximum_speed_limit    float64
 4   weight_restricted      float64
 5   road_type              int64  
 6   start_latitude         float64
 7   start_longitude        float64
 8   start_turn_restricted  object 
 9   end_latitude           float64
 10  end_longitude          float64
 11  end_turn_restricted    object 
dtypes: float64(6), int64(4), object(2)
memory usage: 439.0+ MB


In [11]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV



LR = lgb.LGBMRegressor()


parameters = {'max_depth':[10,15,20],
              'num_leaves':[150, 250,300],
              'learning_rate' : [0.3,0.5, 0.6]}


grid_lr = GridSearchCV(LR, param_grid=parameters, cv=5)


grid_lr.fit(X_train, y_train)


scores_df = pd.DataFrame(grid_lr.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

ValueError: 
All the 135 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\soy66\miniconda3\envs\ds_study\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\soy66\miniconda3\envs\ds_study\lib\site-packages\lightgbm\sklearn.py", line 895, in fit
    super().fit(X, y, sample_weight=sample_weight, init_score=init_score,
  File "C:\Users\soy66\miniconda3\envs\ds_study\lib\site-packages\lightgbm\sklearn.py", line 748, in fit
    self._Booster = train(
  File "C:\Users\soy66\miniconda3\envs\ds_study\lib\site-packages\lightgbm\engine.py", line 271, in train
    booster = Booster(params=params, train_

In [ ]:
grid_lr.cv_results_

In [ ]:
# grid_dtree.cv_results_
LR = lgb.LGBMRegressor(random_state = 42,
                           max_depth = 30,
                           num_leaves = 187,
                           learning_rate = 0.6,
                           n_estimators = 177,
                           reg_lambda = 0.01
                          ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)
print(mae, LR.feature_importances_)